In [ ]:
## Create German text files:
import csv
import numpy as np
import pandas as pd

texts = ['b0', 'b1', 'b2', 'b3', 'b4', 'b5', 'p0', 'p1', 'p2', 'p3', 'p4', 'p5']
strings = []
for text in texts:
  filename = "/content/texts_tags/" + text + '.tags'
  wordTags = pd.read_csv(
      filename,
      sep='\t',
      na_values=['None', '.', 'NA', "NaN",],
      usecols=['WORD', 'WordIndexInSentence', 'STTS_Punctuationbefore', 'STTS_Punctuationafter'])

  sent = ""
  for index, row in wordTags.iterrows():
    if row['WordIndexInSentence'] == 1 and len(strings) != 0:
      strings.append(sent)
      sent = ""

    before = ""
    after = ""

    if row['STTS_Punctuationbefore'] == row['STTS_Punctuationbefore']:
      before = str(row['STTS_Punctuationbefore'])
      before = "".join([before[i] for i in list(range(1, len(before), 2))])

    if row['STTS_Punctuationafter'] == row['STTS_Punctuationafter']:
      after = str(row['STTS_Punctuationafter'])
      after = "".join([after[i] for i in list(range(1, len(after), 2))])
      after = after.replace('(', ')')

    sent += before + str(row["WORD"]) + after + " "
  strings.append(sent)

In [ ]:
readers = [str(i) for i in range(106)]
num_readers = 75
all_words = []
for text in texts:
  words = []
  for reader in readers:
    filename = "/content/readings/reader" + reader + "_" + text + "_merged.txt"
    try:
      reading = pd.read_csv(
          filename,
          sep='\t',
          na_values=['None', '.', 'NA', "NaN",],
          usecols=['WORD', 'WordIndexInSentence', 'WordLen', "FRT", "TFT"])
    except FileNotFoundError:
      continue

    for index, row in reading.iterrows():
      if index > len(words) - 1:
        words.append([
          str(row["WORD"]),
          row['WordIndexInSentence'],
          row['WordLen'],
          row['FRT'] / num_readers,
          row['TFT'] / num_readers,
        ])
      else:
        words[index][3] += row['FRT'] / num_readers
        words[index][4] += row['TFT'] / num_readers
  all_words.extend(words)

In [ ]:
with open("raw_german_texts.txt", "w") as f:
  strings = [string + "\n" for string in strings]
  f.writelines(strings)

with open("german_table.csv", 'w', newline='') as f:
  csvwriter = csv.writer(f)
  csvwriter.writerow(["word", "sentpos", "wlen", "firsttime", "totaltime"])
  csvwriter.writerows(all_words)

In [ ]:
import csv

def create_files(file1, file2, final_RTs, final_text):
  with open(file1, 'r') as f:
    csvreader = csv.reader(f)
    words = {}
    rows = [[r if r != "." else 0 for r in row] for row in csvreader]
    rows.pop(0)

  for row in rows:
    if row[9] not in words.keys():
      words[row[9]] = []
    words[row[9]].append([row[10], len(row[10]), float(row[28]), float(row[53])])

  for key in words.keys():
    first = sum([r[2] for r in words[key]]) / len(words[key])
    total = sum([r[3] for r in words[key]]) / len(words[key])
    words[key] = [words[key][0][0], words[key][0][1], first, total]

  with open(file2, 'r') as f:
    csvreader = csv.reader(f)
    words_list = [r for r in csvreader if r[0] not in ["1-140-27", "2-106-39"]] # removing and cleaning some double-word mistakes in the original data
    words_list.pop(0)

  words_list = [words[row[0]] for row in words_list]

  with open(final_RTs, 'w', newline='') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(["word", "wlen", "firsttime", "totaltime"])
    csvwriter.writerows(words_list)

  with open(file2, 'r') as f:
    csvreader = csv.reader(f)
    rows = [r for r in csvreader if r[0] not in ["1-140-27", "2-106-39"]]
    i = 1
    sents = []
    temp = ""
    while i < len(rows):
      if rows[i][1] != rows[i - 1][1]:
        sents.append(temp)
        temp = "\n"
      temp += words[rows[i][0]][0].strip() + " "
      i += 1
    sents.append(temp)

  with open(final_text, "w") as f:
    f.writelines(sents)

create_files("/content/EnglishReadingData.csv", "/content/EnglishMaterial.xlsx - ALL.csv", "english_RTs.csv", "english_text.txt")
create_files("/content/DutchReadingData.csv", "/content/DutchMaterials.xlsx - ALL.csv", "dutch_RTs.csv", "dutch_text.txt")